# Embedding Watermark


In [10]:
from PIL import Image
import numpy as np
import cv2

### Load the host and watermark images


In [2]:
hr_image = Image.open("host_image.jpg")
wm_image = Image.open("./watermark_image.jpg")

In [3]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)

# Convert images to RGB (if not already in that mode)
hr_image = hr_image.convert("RGB")
wm_image_resized = wm_image_resized.convert("RGB")

# Split images into R, G, B channels
hr_r, hr_g, hr_b = hr_image.split()
wm_r, wm_g, wm_b = wm_image_resized.split()

In [4]:
# Convert channels to numpy arrays
hr_r = np.array(hr_r)
hr_g = np.array(hr_g)
hr_b = np.array(hr_b)
hr_r, hr_g, hr_b

(array([[13, 13, 12, ..., 10, 10, 10],
        [13, 12, 12, ..., 10,  9, 10],
        [13, 12, 12, ..., 10, 10,  9],
        ...,
        [ 0,  0,  0, ...,  1,  1,  1],
        [ 0,  0,  0, ...,  1,  1,  1],
        [ 0,  0,  0, ...,  1,  1,  1]], dtype=uint8),
 array([[19, 19, 18, ..., 43, 43, 42],
        [19, 18, 18, ..., 43, 42, 42],
        [19, 18, 18, ..., 42, 42, 41],
        ...,
        [ 0,  0,  0, ...,  7,  7,  7],
        [ 0,  0,  0, ...,  7,  7,  7],
        [ 0,  0,  0, ...,  7,  7,  7]], dtype=uint8),
 array([[51, 51, 52, ..., 94, 94, 93],
        [51, 50, 52, ..., 94, 93, 93],
        [51, 50, 50, ..., 93, 93, 92],
        ...,
        [ 0,  0,  0, ..., 23, 23, 23],
        [ 0,  0,  0, ..., 23, 23, 23],
        [ 0,  0,  0, ..., 23, 23, 23]], dtype=uint8))

In [5]:
wm_r = np.array(wm_r)
wm_g = np.array(wm_g)
wm_b = np.array(wm_b)
wm_r, wm_g, wm_b

(array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8))

### Function to apply SVD and modify singular values


In [6]:
# Function to apply SVD and modify singular values
def embed_watermark(hr_channel, wm_channel, alpha=0.6):
    U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel)
    U_wm, sigma_wm, V_wm = np.linalg.svd(wm_channel)

    sigma_hr_mod = sigma_hr + alpha * sigma_wm
    hr_channel_mod = np.dot(U_hr[:, :sigma_hr_mod.shape[0]], np.dot(
        np.diag(sigma_hr_mod), V_hr[:sigma_hr_mod.shape[0], :]))

    return hr_channel_mod

### Embed watermark in each channel


In [7]:
hr_r_mod = embed_watermark(hr_r, wm_r)
hr_g_mod = embed_watermark(hr_g, wm_g)
hr_b_mod = embed_watermark(hr_b, wm_b)

In [8]:
# Normalize and convert to uint8
hr_r_mod = cv2.normalize(hr_r_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
hr_g_mod = cv2.normalize(hr_g_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
hr_b_mod = cv2.normalize(hr_b_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
hr_r_mod, hr_g_mod, hr_b_mod

(array([[15, 15, 14, ...,  6,  6,  6],
        [15, 14, 14, ...,  6,  6,  6],
        [15, 14, 14, ...,  6,  6,  6],
        ...,
        [ 2,  2,  3, ...,  1,  1,  1],
        [ 2,  3,  3, ...,  1,  1,  1],
        [ 2,  2,  3, ...,  1,  1,  1]], dtype=uint8),
 array([[22, 22, 22, ..., 30, 30, 30],
        [22, 22, 22, ..., 30, 30, 30],
        [22, 22, 22, ..., 30, 30, 29],
        ...,
        [ 3,  3,  3, ...,  5,  5,  5],
        [ 3,  3,  3, ...,  5,  5,  5],
        [ 3,  3,  3, ...,  5,  5,  5]], dtype=uint8),
 array([[59, 59, 59, ..., 87, 87, 86],
        [59, 58, 59, ..., 87, 86, 86],
        [59, 58, 58, ..., 86, 86, 86],
        ...,
        [ 7,  7,  7, ..., 21, 21, 21],
        [ 7,  7,  7, ..., 21, 21, 21],
        [ 7,  7,  7, ..., 21, 21, 21]], dtype=uint8))

In [9]:
# Merge modified channels back into an RGB image
watermarked_image = Image.merge("RGB", (Image.fromarray(
    hr_r_mod), Image.fromarray(hr_g_mod), Image.fromarray(hr_b_mod)))
watermarked_image.save('output_image-1.png')
watermarked_image.show()

# Extraction


In [27]:
from PIL import Image
import numpy as np
import cv2
# Load the original host and watermarked images
hr_image = Image.open("host_image.jpg").convert("RGB")
watermarked_image = Image.open("output_image-1.png").convert("RGB")
watermarked_image_resized = watermarked_image.resize(hr_image.size)
# Split images into R, G, B channels
hr_r, hr_g, hr_b = hr_image.split()
wm_r, wm_g, wm_b = watermarked_image.split()

# Convert channels to numpy arrays
hr_r = np.array(hr_r)
hr_g = np.array(hr_g)
hr_b = np.array(hr_b)

wm_r = np.array(wm_r)
wm_g = np.array(wm_g)
wm_b = np.array(wm_b)

In [28]:
hr_r.shape

(2794, 5047)

In [35]:
wm_r.shape

(2794, 5047)

In [30]:
hr_r, hr_g, hr_b

(array([[13, 13, 12, ..., 10, 10, 10],
        [13, 12, 12, ..., 10,  9, 10],
        [13, 12, 12, ..., 10, 10,  9],
        ...,
        [ 0,  0,  0, ...,  1,  1,  1],
        [ 0,  0,  0, ...,  1,  1,  1],
        [ 0,  0,  0, ...,  1,  1,  1]], dtype=uint8),
 array([[19, 19, 18, ..., 43, 43, 42],
        [19, 18, 18, ..., 43, 42, 42],
        [19, 18, 18, ..., 42, 42, 41],
        ...,
        [ 0,  0,  0, ...,  7,  7,  7],
        [ 0,  0,  0, ...,  7,  7,  7],
        [ 0,  0,  0, ...,  7,  7,  7]], dtype=uint8),
 array([[51, 51, 52, ..., 94, 94, 93],
        [51, 50, 52, ..., 94, 93, 93],
        [51, 50, 50, ..., 93, 93, 92],
        ...,
        [ 0,  0,  0, ..., 23, 23, 23],
        [ 0,  0,  0, ..., 23, 23, 23],
        [ 0,  0,  0, ..., 23, 23, 23]], dtype=uint8))

In [31]:
wm_r,wm_g,wm_b

(array([[15, 15, 14, ...,  6,  6,  6],
        [15, 14, 14, ...,  6,  6,  6],
        [15, 14, 14, ...,  6,  6,  6],
        ...,
        [ 2,  2,  3, ...,  1,  1,  1],
        [ 2,  3,  3, ...,  1,  1,  1],
        [ 2,  2,  3, ...,  1,  1,  1]], dtype=uint8),
 array([[22, 22, 22, ..., 30, 30, 30],
        [22, 22, 22, ..., 30, 30, 30],
        [22, 22, 22, ..., 30, 30, 29],
        ...,
        [ 3,  3,  3, ...,  5,  5,  5],
        [ 3,  3,  3, ...,  5,  5,  5],
        [ 3,  3,  3, ...,  5,  5,  5]], dtype=uint8),
 array([[59, 59, 59, ..., 87, 87, 86],
        [59, 58, 59, ..., 87, 86, 86],
        [59, 58, 58, ..., 86, 86, 86],
        ...,
        [ 7,  7,  7, ..., 21, 21, 21],
        [ 7,  7,  7, ..., 21, 21, 21],
        [ 7,  7,  7, ..., 21, 21, 21]], dtype=uint8))

In [32]:
# U_hr, sigma_hr, V_hr = np.linalg.svd(hr_r)
U_hr_mod, sigma_hr_mod, V_hr_mod = np.linalg.svd(wm_r)

In [40]:
# Create an identity matrix of the desired shape
diagonal_matrix = np.eye(2794, 5047)

# Set the diagonal values of the identity matrix to sigma_hr_mod
diagonal_matrix[np.arange(2794), np.arange(2794)] = sigma_hr_mod

In [41]:

# Calculate watermark singular values
sigma_wm = (diagonal_matrix - hr_r) / 0.6
sigma_wm
# # Step 3: Reconstruct the watermark channel using the estimated singular values
# # Choose the same U and V used for watermark embedding (assuming they're same for simplicity)
# U_wm, V_wm = U_hr[:sigma_wm.shape[0], :], V_hr_mod[:, :sigma_wm.shape[0]].T

# # Reconstruct wm_channel
# wm_channel = np.dot(U_wm, np.dot(np.diag(sigma_wm), V_wm))


array([[ 6.71010958e+05, -2.16666667e+01, -2.00000000e+01, ...,
        -1.66666667e+01, -1.66666667e+01, -1.66666667e+01],
       [-2.16666667e+01,  8.64224201e+04, -2.00000000e+01, ...,
        -1.66666667e+01, -1.50000000e+01, -1.66666667e+01],
       [-2.16666667e+01, -2.00000000e+01,  5.91148433e+04, ...,
        -1.66666667e+01, -1.66666667e+01, -1.50000000e+01],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.66666667e+00, -1.66666667e+00, -1.66666667e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.66666667e+00, -1.66666667e+00, -1.66666667e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.66666667e+00, -1.66666667e+00, -1.66666667e+00]])

In [23]:
# %% [markdown]
# ## Watermark Extraction

# This code assumes you have access to the watermarked image 
# ('output_image-1.png') and the original host image ('host_image.jpg').

# %%
from PIL import Image
import numpy as np
import cv2

# %%
# Load watermarked and host images
watermarked_image = Image.open("output_image-1.png")
host_image = Image.open("host_image.jpg")

# Convert images to RGB (if not already in that mode)
watermarked_image = watermarked_image.convert("RGB")
host_image = host_image.convert("RGB")

# Split images into R, G, B channels
watermarked_r, watermarked_g, watermarked_b = watermarked_image.split()
host_r, host_g, host_b = host_image.split()

# %%
# Convert channels to numpy arrays
watermarked_r = np.array(watermarked_r)
watermarked_g = np.array(watermarked_g)
watermarked_b = np.array(watermarked_b)
host_r = np.array(host_r)
host_g = np.array(host_g)
host_b = np.array(host_b)

# %%
# Function to apply SVD and extract watermark

def extract_watermark(watermarked_channel, host_channel, alpha=0.6):
  """
  This function extracts the watermark from a watermarked channel.

  Args:
      watermarked_channel: The watermarked channel (numpy array).
      host_channel: The corresponding channel from the host image (numpy array).
      alpha: The alpha value used during embedding (default=0.6).

  Returns:
      watermark_estimate: The estimated watermark channel (numpy array).
  """
  U_wm, sigma_wm, V_wm = np.linalg.svd(watermarked_channel)
  U_host, sigma_host, V_host = np.linalg.svd(host_channel)

  # Estimate watermark singular values
  sigma_watermark_estimate = sigma_wm - alpha * sigma_host

  # Reconstruct watermark using estimated singular values
  watermark_estimate = np.dot(U_wm[:, :sigma_watermark_estimate.shape[0]], 
                             np.dot(np.diag(sigma_watermark_estimate), V_wm[:sigma_watermark_estimate.shape[0], :]))
  
  return watermark_estimate

# %%
# Extract watermark from each channel
watermark_r_estimate = extract_watermark(watermarked_r, host_r)
watermark_g_estimate = extract_watermark(watermarked_g, host_g)
watermark_b_estimate = extract_watermark(watermarked_b, host_b)

# %%
# Normalize and convert to uint8 (might require adjustments based on watermark properties)
watermark_r_estimate = cv2.normalize(watermark_r_estimate, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
watermark_g_estimate = cv2.normalize(watermark_g_estimate, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
watermark_b_estimate = cv2.normalize(watermark_b_estimate, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# Merge estimated watermark channels
estimated_watermark = Image.merge("RGB", (Image.fromarray(watermark_r_estimate), 
                                          Image.fromarray(watermark_g_estimate), 
                                          Image.fromarray(watermark_b_estimate)))

# Save and potentially display the estimated watermark
estimated_watermark.save("extracted_watermark.png")
# estimated_watermark.show()  # Uncomment to display the estimated watermark

print("Watermark extraction complete! Check 'extracted_watermark.png'")

Watermark extraction complete! Check 'extracted_watermark.png'


In [24]:
from PIL import Image
import numpy as np
import cv2

KeyboardInterrupt: 

### Load the host,watermark and watermarked images

In [ ]:
hr_image = Image.open("./output_image-1.png")
wm_image = Image.open("./watermark_image.jpg")
watermarked_image = Image.open("./output_image-1.png")

# hr_image = Image.open("host_image.jpg")
# wm_image = Image.open("./watermark_image.jpg")
# watermarked_image = Image.open("./output_image-1.png")

In [ ]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)
watermarked_image_resized = watermarked_image.resize(hr_image.size)

# Convert images to RGB (if not already in that mode)
hr_image = hr_image.convert("RGB")
wm_image_resized = wm_image_resized.convert("RGB")
watermarked_image_resized = watermarked_image_resized.convert("RGB")

# Split images into R, G, B channels
hr_r, hr_g, hr_b = hr_image.split()
wm_r, wm_g, wm_b = wm_image_resized.split()
hr_r_mod, hr_g_mod, hr_b_mod = watermarked_image_resized.split()

In [ ]:
# Convert channels to numpy arrays
hr_r = np.array(hr_r)
hr_g = np.array(hr_g)
hr_b = np.array(hr_b)
hr_r, hr_g, hr_b

(array([[15, 15, 14, ...,  6,  6,  6],
        [15, 14, 14, ...,  6,  6,  6],
        [15, 14, 14, ...,  6,  6,  6],
        ...,
        [ 2,  2,  3, ...,  1,  1,  1],
        [ 2,  3,  3, ...,  1,  1,  1],
        [ 2,  2,  3, ...,  1,  1,  1]], dtype=uint8),
 array([[22, 22, 22, ..., 30, 30, 30],
        [22, 22, 22, ..., 30, 30, 30],
        [22, 22, 22, ..., 30, 30, 29],
        ...,
        [ 3,  3,  3, ...,  5,  5,  5],
        [ 3,  3,  3, ...,  5,  5,  5],
        [ 3,  3,  3, ...,  5,  5,  5]], dtype=uint8),
 array([[59, 59, 59, ..., 87, 87, 86],
        [59, 58, 59, ..., 87, 86, 86],
        [59, 58, 58, ..., 86, 86, 86],
        ...,
        [ 7,  7,  7, ..., 21, 21, 21],
        [ 7,  7,  7, ..., 21, 21, 21],
        [ 7,  7,  7, ..., 21, 21, 21]], dtype=uint8))

In [ ]:
wm_r = np.array(wm_r)
wm_g = np.array(wm_g)
wm_b = np.array(wm_b)
wm_r, wm_g, wm_b

(array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8))

In [ ]:
hr_r_mod = np.array(hr_r_mod)
hr_g_mod = np.array(hr_g_mod)
hr_b_mod = np.array(hr_b_mod)
hr_r_mod, hr_g_mod, hr_b_mod

(array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8),
 array([[255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]], dtype=uint8))

### Extraction process

In [ ]:
import numpy as np
from tqdm import tqdm

def extract_watermark(hr_mod_channel, hr_channel, wr_channel, alpha=0.6):
    # Initialize the progress bar
    with tqdm(total=100, desc=f"Extracting Watermark-{hr_channel}", bar_format="{l_bar}{bar} [ time left: {remaining} ]") as pbar:
        # Perform SVD on the modified HR channel
        U_hr_mod, sigma_hr_mod, V_hr_mod = np.linalg.svd(hr_mod_channel, full_matrices=False)
        pbar.update(30)  # Update progress bar
        
        # Perform SVD on the original HR channel
        U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel, full_matrices=False)
        pbar.update(30)  # Update progress bar
        
        # Perform SVD on the WR channel
        U_wm, sigma_wm, V_wm = np.linalg.svd(wr_channel, full_matrices=False)
        pbar.update(30)  # Update progress bar

        # Calculate the modified watermark channel
        sigma_wm_mod = (sigma_hr_mod - sigma_hr) / alpha
        wm_channel_mod = np.dot(U_wm, np.dot(np.diag(sigma_wm_mod), V_wm))
        pbar.update(10)  # Final progress update
    
    return wm_channel_mod


### Extract watermark from each channel

In [ ]:
wm_r_mod = extract_watermark(hr_r_mod, hr_r, wm_r)
wm_g_mod = extract_watermark(hr_g_mod, hr_g, wm_g)
wm_b_mod = extract_watermark(hr_b_mod, hr_b, wm_b)

Extracting Watermark-{hr_channel}: 100%|██████████ [ time left: 00:00 ]
Extracting Watermark-{hr_channel}: 100%|██████████ [ time left: 00:00 ]
Extracting Watermark-{hr_channel}: 100%|██████████ [ time left: 00:00 ]


In [ ]:
# Normalize and convert to uint8
wm_r_mod = cv2.normalize(wm_r_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
wm_g_mod = cv2.normalize(wm_g_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
wm_b_mod = cv2.normalize(wm_b_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
wm_r_mod, wm_g_mod, wm_b_mod

(array([[206, 204, 180, ..., 202, 201, 204],
        [215, 135, 204, ..., 204, 204, 204],
        [215, 202, 182, ..., 205, 205, 204],
        ...,
        [222, 207, 207, ..., 205, 205, 204],
        [222, 208, 208, ..., 204, 204, 204],
        [222, 208, 207, ..., 204, 205, 204]], dtype=uint8),
 array([[183, 184, 165, ..., 180, 180, 180],
        [170, 242, 169, ..., 179, 179, 179],
        [181, 191, 208, ..., 179, 179, 179],
        ...,
        [196, 182, 178, ..., 178, 178, 179],
        [196, 179, 180, ..., 179, 179, 179],
        [196, 179, 180, ..., 180, 180, 179]], dtype=uint8),
 array([[165, 161, 163, ..., 160, 159, 162],
        [162, 100, 168, ..., 163, 163, 163],
        [162, 155, 168, ..., 162, 161, 163],
        ...,
        [179, 163, 163, ..., 163, 162, 163],
        [179, 165, 162, ..., 162, 163, 163],
        [179, 165, 162, ..., 163, 164, 163]], dtype=uint8))

In [ ]:
# Merge extracted channels back into an RGB image
extracted_watermark = Image.merge("RGB", (Image.fromarray(
    wm_r_mod), Image.fromarray(wm_g_mod), Image.fromarray(wm_b_mod)))
extracted_watermark.save('watermark_output_image-1.png')
extracted_watermark.show()